In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
import pandas as pd
import math
import nltk
from nltk.corpus import stopwords
import string
from nltk import word_tokenize
nltk.download('words')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm,tqdm_pandas
nltk.download('brown')
from nltk.corpus import brown

words = brown.words(categories="news")
sents = brown.sents(categories=["news"])

print("Number of words: ",len(words))
print("Number of sentences: ",len(sents))
print("Number of unique words (including capitalized)",
      len(set(words)))

[nltk_data] Downloading package words to /home/yui/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /home/yui/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/yui/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package brown to /home/yui/nltk_data...
[nltk_data]   Package brown is already up-to-date!


Number of words:  100554
Number of sentences:  4623
Number of unique words (including capitalized) 14394
